In [25]:
import time

from enum import Enum
class OrderType(Enum):
    LIMIT = 1
    MARKET = 2
    IOC = 3

class OrderSide(Enum):
    BUY = 1
    SELL = 2


class NonPositiveQuantity(Exception):
    pass

class NonPositivePrice(Exception):
    pass

class InvalidSide(Exception):
    pass

class UndefinedOrderType(Exception):
    pass

class UndefinedOrderSide(Exception):
    pass

class NewQuantityNotSmaller(Exception):
    pass

class UndefinedTraderAction(Exception):
    pass

class UndefinedResponse(Exception):
    pass


from abc import ABC


class Order(ABC):
    def __init__(self, id, symbol, quantity, side, time):
        self.id = id
        self.symbol = symbol
        if quantity > 0:
            self.quantity = quantity
        else:
            raise NonPositiveQuantity("Quantity Must Be Positive!")
        if side in [OrderSide.BUY, OrderSide.SELL]:
            self.side = side
        else:
            raise InvalidSide("Side Must Be Either \"Buy\" or \"OrderSide.SELL\"!")
        self.time = time


class LimitOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.LIMIT


class MarketOrder(Order):
    def __init__(self, id, symbol, quantity, side, time):
        super().__init__(id, symbol, quantity, side, time)
        self.type = OrderType.MARKET


class IOCOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.IOC
    

class FilledOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time, limit = False):
        super().__init__(id, symbol, quantity, side, time)
        self.price = price
        self.limit = limit
        



class MatchingEngine():
    def __init__(self):
        self.bid_book = []
        self.ask_book = []
    
    def insert_limit_order(self, order):
        assert order.type == OrderType.LIMIT
        # Implement this function
        # this function's sole puporse is to place limit orders in the book that are guaranteed
        # to not immediately fill
        # You need to raise the following error if the side the order is for is ambiguous
        if order.side == OrderSide.BUY:
            self.bid_book.append(order)
            self.bid_book.sort(key=lambda x: x.price, reverse=True)
        elif order.side == OrderSide.SELL:
            self.ask_book.append(order)
            self.ask_book.sort(key=lambda x: x.price)
        else:
            raise UndefinedOrderSide("Undefined Order Side!")

    def handle_order(self, order):
        # Implement this function
        # In this function you need to call different functions from the matching engine
        # depending on the type of order you are given
        if order.type == OrderType.LIMIT:
            return self.handle_limit_order(order)
        elif order.type == OrderType.MARKET:
            return self.handle_market_order(order)
        elif order.type == OrderType.IOC:
            return self.handle_ioc_order(order)
        else:
        # You need to raise the following error if the type of order is ambiguous
            raise UndefinedOrderType("Undefined Order Type!")
    
    def handle_limit_order(self, order): 
        # Implement this function
        # Keep in mind what happens to the orders in the limit order books when orders get filled
        # or if there are no crosses from this order
        # in other words, handle_limit_order accepts an arbitrary limit order that can either be 
        # filled if the limit order price crosses the book, or placed in the book. If the latter, 
        # pass the order to insert_limit_order below. 
        filled_orders = []
        # The orders that are filled from the market order need to be inserted into the above list
        if order.side == OrderSide.BUY:
            matching_book = self.ask_book
        elif order.side == OrderSide.SELL:
            matching_book = self.bid_book
        else:
        # You need to raise the following error if the side the order is for is ambiguous
            raise UndefinedOrderSide("Undefined Order Side!")

        i = 0
        while i < len(matching_book) and order.quantity > 0:
            match = matching_book[i]
            if (order.side == OrderSide.BUY and order.price >= match.price) or \
               (order.side == OrderSide.SELL and order.price <= match.price):
                fill_quantity = min(order.quantity, match.quantity)

                # Create filled order records for the matched order
                filled_order_match = FilledOrder(match.id, match.symbol, fill_quantity, match.price, match.side, match.time, limit=True)
                filled_orders.append(filled_order_match)

                # Create a filled order record for the portion of the incoming order
                filled_order_incoming = FilledOrder(order.id, order.symbol, fill_quantity, match.price, order.side, order.time, limit=True)
                filled_orders.append(filled_order_incoming)

                # Adjusting quantities
                order.quantity -= fill_quantity
                match.quantity -= fill_quantity

                # Remove the matching order from the book if it's fully filled
                if match.quantity == 0:
                    matching_book.pop(i)
                else:
                    i += 1
            else:
                i += 1

        if order.quantity > 0:
            self.insert_limit_order(order)
        
        # The filled orders are expected to be the return variable (list)
        return filled_orders
        
    
    def handle_market_order(self, order):
        # Implement this function
        filled_orders = []
        # The orders that are filled from the market order need to be inserted into the above list
        if order.side == OrderSide.BUY:
            matching_book = self.ask_book
        elif order.side == OrderSide.SELL:
            matching_book = self.bid_book
        else:
        # You need to raise the following error if the side the order is for is ambiguous
            raise UndefinedOrderSide("Undefined Order Side!")
        
        for i in range(len(matching_book)):
            if order.side == OrderSide.BUY:
                if order.quantity >= matching_book[i].quantity:
                    filled_orders.append(FilledOrder(matching_book[i].id, matching_book[i].symbol, matching_book[i].quantity, matching_book[i].price, matching_book[i].side, matching_book[i].time))
                    order.quantity -= matching_book[i].quantity
                    matching_book[i].quantity = 0
                else:
                    filled_orders.append(FilledOrder(matching_book[i].id, matching_book[i].symbol, order.quantity, matching_book[i].price, matching_book[i].side, matching_book[i].time))
                    matching_book[i].quantity -= order.quantity
                    order.quantity = 0
                    break
            elif order.side == OrderSide.SELL:
                if order.quantity >= matching_book[i].quantity:
                    filled_orders.append(FilledOrder(matching_book[i].id, matching_book[i].symbol, matching_book[i].quantity, matching_book[i].price, matching_book[i].side, matching_book[i].time))
                    order.quantity -= matching_book[i].quantity
                    matching_book[i].quantity = 0
                else:
                    filled_orders.append(FilledOrder(matching_book[i].id, matching_book[i].symbol, order.quantity, matching_book[i].price, matching_book[i].side, matching_book[i].time))
                    matching_book[i].quantity -= order.quantity
                    order.quantity = 0
                    break
        # The filled orders are expected to be the return variable (list)
        return filled_orders
        

    def handle_ioc_order(self, order):
        # Implement this function
        filled_orders = []
        # The orders that are filled from the ioc order need to be inserted into the above list
        if order.side == OrderSide.BUY:
            matching_book = self.ask_book
        elif order.side == OrderSide.SELL:
            matching_book = self.bid_book
        else:
        # You need to raise the following error if the side the order is for is ambiguous
            raise UndefinedOrderSide("Undefined Order Side!")
        # The filled orders are expected to be the return variable (list)
        for i in range(len(matching_book)):
            if order.side == OrderSide.BUY:
                if order.price >= matching_book[i].price:
                    if order.quantity >= matching_book[i].quantity:
                        filled_orders.append(FilledOrder(matching_book[i].id, matching_book[i].symbol, matching_book[i].quantity, matching_book[i].price, matching_book[i].side, matching_book[i].time, True))
                        order.quantity -= matching_book[i].quantity
                        matching_book[i].quantity = 0
                    else:
                        filled_orders.append(FilledOrder(matching_book[i].id, matching_book[i].symbol, order.quantity, matching_book[i].price, matching_book[i].side, matching_book[i].time, True))
                        matching_book[i].quantity -= order.quantity
                        order.quantity = 0
                        break
            elif order.side == OrderSide.SELL:
                if order.price <= matching_book[i].price:
                    if order.quantity >= matching_book[i].quantity:
                        filled_orders.append(FilledOrder(matching_book[i].id, matching_book[i].symbol, matching_book[i].quantity, matching_book[i].price, matching_book[i].side, matching_book[i].time, True))
                        order.quantity -= matching_book[i].quantity
                        matching_book[i].quantity = 0
                    else:
                        filled_orders.append(FilledOrder(matching_book[i].id, matching_book[i].symbol, order.quantity, matching_book[i].price, matching_book[i].side, matching_book[i].time, True))
                        matching_book[i].quantity -= order.quantity
                        order.quantity = 0
                        break
        return filled_orders
        
    def amend_quantity(self, id, quantity):
        # Implement this function
        # Hint: Remember that there are two order books, one on the bid side and one on the ask side
        if quantity > 0:
            for order in self.bid_book:
                if order.id == id:
                    if quantity < order.quantity:
                        order.quantity = quantity
                        return True
                    else:
                        raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!")
                        return False
            for order in self.ask_book:
                if order.id == id:
                    if quantity < order.quantity:
                        order.quantity = quantity
                        return True
                    else:
        # You need to raise the following error if the user attempts to modify an order
        # with a quantity that's greater than given in the existing order
                        raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!")
                        return False
        else:
            raise NonPositiveQuantity("Quantity Must Be Positive!")
            return False

    def cancel_order(self, id):
        # Implement this function
        # Think about the changes you need to make in the order book based on the parameters given
        for order in self.bid_book:
            if order.id == id:
                self.bid_book.remove(order)
                return True
        for order in self.ask_book:
            if order.id == id:
                self.ask_book.remove(order)
                return True
        return False

#test cases to see if i passed
import unittest

class TestOrderBook(unittest.TestCase):

    def test_insert_limit_order(self):
        matching_engine = MatchingEngine()
        order = LimitOrder(1, "S", 10, 10, OrderSide.BUY, time.time())
        matching_engine.insert_limit_order(order)

        self.assertEqual(matching_engine.bid_book[0].quantity, 10)
        self.assertEqual(matching_engine.bid_book[0].price, 10)
    
    def test_handle_limit_order(self):
        matching_engine = MatchingEngine()
        order = LimitOrder(1, "S", 10, 10, OrderSide.BUY, time.time())
        matching_engine.insert_limit_order(order)

        order_1 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(3, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        self.assertEqual(matching_engine.bid_book[0].price, 15)
        self.assertEqual(matching_engine.bid_book[1].quantity, 10)

        order_sell = LimitOrder(4, "S", 14, 8, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_limit_order(order_sell)

        self.assertEqual(matching_engine.bid_book[0].quantity, 6)
        self.assertEqual(filled_orders[0].id, 3)
        self.assertEqual(filled_orders[0].price, 15)
        self.assertEqual(filled_orders[2].id, 1)
        self.assertEqual(filled_orders[2].price, 10)
    
    def test_handle_market_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 6, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        order = MarketOrder(5, "S", 5, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_market_order(order)
        self.assertEqual(matching_engine.bid_book[0].quantity, 1)
        self.assertEqual(filled_orders[0].price, 10)

    def test_handle_ioc_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 1, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        order = IOCOrder(6, "S", 5, 12, OrderSide.SELL, time.time())
        filled_orders = matching_engine.handle_ioc_order(order)
        self.assertEqual(matching_engine.bid_book[0].quantity, 1)
        self.assertEqual(len(filled_orders), 0)
    
    def test_amend_quantity(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        matching_engine.amend_quantity(2, 8)
        self.assertEqual(matching_engine.bid_book[0].quantity, 8)
    
    def test_cancel_order(self):
        matching_engine = MatchingEngine()
        order_1 = LimitOrder(1, "S", 5, 10, OrderSide.BUY, time.time())
        order_2 = LimitOrder(2, "S", 10, 15, OrderSide.BUY, time.time())
        matching_engine.handle_limit_order(order_1)
        matching_engine.handle_limit_order(order_2)

        matching_engine.cancel_order(1)
        self.assertEqual(matching_engine.bid_book[0].id, 2)

import io
import __main__
suite = unittest.TestLoader().loadTestsFromModule(__main__)
buf = io.StringIO()
unittest.TextTestRunner(stream=buf, verbosity=2).run(suite)
buf = buf.getvalue().split("\n")
for test in buf:
	if test.startswith("test"):
		print(test)

test_amend_quantity (__main__.TestOrderBook.test_amend_quantity) ... ok
test_cancel_order (__main__.TestOrderBook.test_cancel_order) ... ok
test_handle_ioc_order (__main__.TestOrderBook.test_handle_ioc_order) ... ok
test_handle_limit_order (__main__.TestOrderBook.test_handle_limit_order) ... ok
test_handle_market_order (__main__.TestOrderBook.test_handle_market_order) ... ok
test_insert_limit_order (__main__.TestOrderBook.test_insert_limit_order) ... ok


In [18]:
matching_engine = MatchingEngine()
order = LimitOrder(1, "S", 10, 10, OrderSide.BUY, time.time())
matching_engine.insert_limit_order(order)

order_1 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
order_2 = LimitOrder(3, "S", 10, 15, OrderSide.BUY, time.time())
matching_engine.handle_limit_order(order_1)
matching_engine.handle_limit_order(order_2)

[]

In [21]:
for i in matching_engine.bid_book:
    print(i.id, i.quantity, i.price)

3 10 15
1 10 10
2 5 10


In [22]:
order_sell = LimitOrder(4, "S", 14, 8, OrderSide.SELL, time.time())
filled_orders = matching_engine.handle_limit_order(order_sell)

In [23]:
for i in filled_orders:
    print(i.id, i.quantity, i.price)

3 10 15
1 4 10
4 4 10


In [24]:
for i in matching_engine.bid_book:
    print(i.id, i.quantity, i.price)

1 6 10
2 5 10
